In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
from distgen.parsing import is_floatable


def test_isfloatable():
    assert not is_floatable("foobar")
    assert not is_floatable([])
    assert not is_floatable({})

    assert is_floatable(1)
    assert is_floatable("1234")
    assert is_floatable(12350.0)


test_isfloatable()

In [ ]:
from distgen.parsing import is_unit
from distgen.physical_constants import unit_registry


def test_is_unit():
    # Strings with a value and unit return False
    assert not is_unit("2 ps")
    assert not is_unit("1 ps")
    assert not is_unit("1.0 ps")

    # Strings with known units work, as will an actual Pint unit object
    assert is_unit("ps")
    assert is_unit(unit_registry.Unit("ps"))


test_is_unit()

In [ ]:
from distgen.parsing import is_quantizable


def test_is_quantizable():
    assert not is_quantizable(10.0)
    assert not is_quantizable("10.0")
    assert not is_quantizable({"value": 120.1})
    assert not is_quantizable({"units": 120.1})

    Q_ = unit_registry.Quantity
    assert is_quantizable(Q_(10, "ps"))
    assert is_quantizable("10 ps")
    assert is_quantizable({"value": 120.1, "units": "mm"})


test_is_quantizable()

In [ ]:
from distgen.parsing import dict_to_quantity


def test_dict_to_quantity():
    q1 = dict_to_quantity({"value": 102.1, "units": "eV"})
    q2 = unit_registry.Quantity(102.1, "eV")

    assert q1 == q2


test_dict_to_quantity()

In [ ]:
from distgen.parsing import parse_quantity


def test_parse_quantity():
    q1 = unit_registry.Quantity(102.1, "eV")
    q2 = parse_quantity("102.1 eV")
    q3 = parse_quantity({"value": 102.1, "units": "eV"})

    assert q1 == q2
    assert q2 == q3


test_parse_quantity()

In [ ]:
from distgen.parsing import update_quantity
from pint import DimensionalityError


def test_update_quantity():
    qi = unit_registry.Quantity(102.1, "eV")

    try:
        update_quantity(qi, "10 ps")

    except Exception as ex:
        if not isinstance(ex, DimensionalityError):
            raise ValueError("Didn't catch dimensionality error")

    assert update_quantity(qi, 10) == unit_registry.Quantity(10, "eV")
    assert update_quantity(qi, "10 eV") == unit_registry.Quantity(10, "eV")
    assert update_quantity(qi, {"value": 10, "units": "eV"}) == unit_registry.Quantity(
        10, "eV"
    )


test_update_quantity()

In [ ]:
from distgen.parsing import update_quantity_in_dict


def test_update_quantity_in_dict():
    d = {"x": unit_registry.Quantity("10 ps")}
    # d2 = {"x": unit_registry.Quantity("12 ps")}

    try:
        update_quantity_in_dict("x", d, "10 mm")

    except Exception as ex:
        if not isinstance(ex, DimensionalityError):
            raise ValueError("Didn't catch dimensionality error")

    update_quantity_in_dict("x", d, "12 ps")
    assert d == {"x": unit_registry.Quantity("12 ps")}

    update_quantity_in_dict("x", d, 13)
    assert d == {"x": unit_registry.Quantity("13 ps")}

    update_quantity_in_dict("x", d, unit_registry.Quantity("14 ps"))
    assert d == {"x": unit_registry.Quantity("14 ps")}

    update_quantity_in_dict("x", d, {"value": 15, "units": "ps"})


test_update_quantity_in_dict()

In [ ]:
from distgen.parsing import list_to_dict


def test_list_to_dict():
    list_ex = ["x", unit_registry.Quantity(10, "s"), 10]
    d = list_to_dict(list_ex)
    assert d == {0: "x", 1: unit_registry.Quantity(10, "s"), 2: 10}


test_list_to_dict()

In [ ]:
from distgen.parsing import convert_list_quantities


def test_convert_list_quantities():
    list_ex = ["10 ps", {"value": 12, "units": "ps"}]
    convert_list_quantities(list_ex)
    assert list_ex == [
        unit_registry.Quantity(10, "ps"),
        unit_registry.Quantity(12, "ps"),
    ]


test_convert_list_quantities()